In [1]:
import os
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import AIMessage, HumanMessage, AnyMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition

In [2]:
load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b

tools = [add,multiply,divide]

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key)

llm_with_tools = llm.bind_tools(tools,parallel_tool_calls=False)

sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

In [5]:
def assistant(state: MessagesState):
       return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

builder = StateGraph(MessagesState)

builder.add_node("assistant", assistant)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools","assistant")
react_graph = builder.compile()

messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (11ff673b-78fd-4e50-aedc-9fa30b86d468)
 Call ID: 11ff673b-78fd-4e50-aedc-9fa30b86d468
  Args:
    b: 4
    a: 3
  multiply (5f1c04ef-4836-41b4-aa6d-1014c7ed2c1e)
 Call ID: 5f1c04ef-4836-41b4-aa6d-1014c7ed2c1e
  Args:
    b: 2
    a: 7
  divide (2cae3249-bb92-4444-9ac3-f26423ddce82)
 Call ID: 2cae3249-bb92-4444-9ac3-f26423ddce82
  Args:
    b: 5
    a: 14
================================= Tool Message =================================
Name: add

7
================================= Tool Message =================================
Name: multiply

14
================================= Tool Message =================================
Name: divide

2.8
================================== Ai Message ==================================

The sum of 3 and 4 is 7. 